In [2]:
import pandas as pd
import re
import wordninja
from pypdf import PdfReader
from pathlib import Path

dir_path = Path()
bini = dir_path/"bini_dict.pdf"

pdf = PdfReader(bini)
pdf_19 = pdf.pages[19].extract_text()
print(pdf_19)

abokpoi["J(i)"woof"made
ofwood,'usedinweaving
;
strikesthethreadsdowninordertofastenthem.(2)astickheldbywomenataburialorsecondburial,representingamatchet;itissupposedtokeep
evilspiritsawayfromthedeceased'schildren.abokpo2['*.]atree,Diospyros
crassiflora,usedinbuildinghousesandalso(bytheJskriandIjopeople)forpaddles(abokpo1isnotmadefrom
it).
abutete[.'..](high)edgeofaroad
;
v.aze[.'].
ada["]state-sword,wornbytheDba,somebigchiefs,andthepriestsofOsa[..]andOxwahs
[J.Y,of.Yor.ada[./].ada[/]family-representativesat
aburial.ada[..]junction;crossroads.Adabi"'[.).]adeitysupposedtostandontheboundarybetweentheworldandsfioi[.'.],onAd-agb-ad-sfim[.'.'•.]:therethedeadpeoplerestawhile.AnAdabiisalsomadeandwor-shippedbythepriestessesof
Oloka["•];cfada[..].adeke[. ]femaleofosels[.](?);
itcarriesmanyeggs;lookslike
themaleogoro[..'],buthasshiningstripes;cfeks[."%].
adese[*
](1)middle;ades-ot-
oye['X']{tisinthemiddle
(ofit);ades-uhuou[.*.'.]crown
ofhead.(2)adessn-eva
[/J"themiddleoftwo":be-tween;ysl-adessn-

the bini dictionary starts from page 18 down to page 251 for all the word and translations used to form the coropra, the latter pages may contain some other useful information that can be used in some manner to help in translations and effective corpora development 

In [3]:
def read_text(start: int, file, folder):
    """
    **writes all pages of the pdf into a single text file**
    """
    file_path = Path(file)
    doc = PdfReader(file_path)
    folder_path = Path(folder)

    if folder_path.exists():
        pass
    else:
        folder_path.mkdir()

    with open(f"{folder_path}/raw.txt", "w") as f:
        for page in range(start, doc.get_num_pages()):
            f.write(doc.pages[page].extract_text(extraction_mode='plain')) # chnage the extraction_mode argument to layout for different extraction, makes it harder to clean imo
        
    
file = dir_path/"bini_dict.pdf"
folder = dir_path/"raw"
read_text(start=18, file=file, folder=folder)

In [4]:
def clean_text(text):
    # Remove special annotation symbols (e.g., [.*], [..], etc.)
    text = re.sub(r'\[[^\]]*\]', '', text)
    # Replace line breaks within words with a space
    text = re.sub(r'([a-zA-Z0-9])\n([a-zA-Z0-9])', r'\1 \2', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)
    # Attempt to split merged words using wordninja
    words = []
    for token in text.split():
        if len(token) > 15:  # likely merged
            words.extend(wordninja.split(token))
        else:
            words.append(token)
    return ' '.join(words)

def clean_bini_dictionary(raw_text):
    # Split the text into lines
    lines = raw_text.strip().split('\n')

    cleaned_entries = []
    entry = ""

    for line in lines:
        line = line.strip()

        # If a line starts with a lowercase word followed by some symbol (likely a new entry)
        if re.match(r"^[a-zA-Z]+\s?[\[\(']", line):
            # Save the previous entry if exists
            if entry:
                cleaned_entries.append(entry.strip())
                entry = ""

            # Start new entry
            parts = re.split(r'\s*[\[\(].*?\]\s*', line, maxsplit=1)
            if len(parts) >= 2:
                bini_word = parts[0].strip()
                definition = parts[1].strip()
                entry = f"{bini_word} – {definition}"
            else:
                entry = line  # fallback if format is unexpected
        else:
            # Append continuation lines
            entry += " " + line.strip()

    # Add the last entry
    if entry:
        cleaned_entries.append(entry.strip())

    return '\n\n'.join(cleaned_entries)


# Example usage
with open('raw/raw.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()

cleaned_text = clean_bini_dictionary(raw_text)

with open('bini_cleaned.txt', 'w', encoding='utf-8') as f:
    f.write(cleaned_text)

print("Cleaning done. Output saved to bini_cleaned.txt.")


Cleaning done. Output saved to bini_cleaned.txt.


#### plan A:
- use regex to write the texts so that the bini words always start a new line
- find the pattern for whatever comes before the bini word and whatever comes after the bini word
- extract the bini words and store them in a df and also store the definitions in a df, 
- probably use gpt to generate a translation from the definitions of the words
- save that as the corpus

In [ ]:
from unstructured.cleaners.core import clean, clean_extra_whitespace
from unstructured.cleaners.extract import extract_text_after, extract_text_before

u19 = extract_text_after(pdf_19, pattern=r"\w+\[")


u19

'"J(i)"woof"made\nofwood,\'usedinweaving\n;\nstrikesthethreadsdowninordertofastenthem.(2)astickheldbywomenataburialorsecondburial,representingamatchet;itissupposedtokeep\nevilspiritsawayfromthedeceased\'schildren.abokpo2[\'*.]atree,Diospyros\ncrassiflora,usedinbuildinghousesandalso(bytheJskriandIjopeople)forpaddles(abokpo1isnotmadefrom\nit).\nabutete[.\'..](high)edgeofaroad\n;\nv.aze[.\'].\nada["]state-sword,wornbytheDba,somebigchiefs,andthepriestsofOsa[..]andOxwahs\n[J.Y,of.Yor.ada[./].ada[/]family-representativesat\naburial.ada[..]junction;crossroads.Adabi"\'[.).]adeitysupposedtostandontheboundarybetweentheworldandsfioi[.\'.],onAd-agb-ad-sfim[.\'.\'•.]:therethedeadpeoplerestawhile.AnAdabiisalsomadeandwor-shippedbythepriestessesof\nOloka["•];cfada[..].adeke[. ]femaleofosels[.](?);\nitcarriesmanyeggs;lookslike\nthemaleogoro[..\'],buthasshiningstripes;cfeks[."%].\nadese[*\n](1)middle;ades-ot-\noye[\'X\']{tisinthemiddle\n(ofit);ades-uhuou[.*.\'.]crown\nofhead.(2)adessn-eva\n[/J"themiddle